In [1]:
# Enable autoreload of module
%load_ext autoreload
%autoreload 2

import pandas as pd
import torch
import json
import os
from data.DWSNets_dataset import DWSNetsDataset
from networks.regression_transformer import RegressionTransformerConfig, RegressionTransformer
from training.naive_approach import train, Config


In [2]:
# Config
config = Config()
config.learning_rate=4e-3
config.weight_decay=0
config.decay_lr=True
config.lr_decay_iters=60000
config.warmup_iters=0
model_config = RegressionTransformerConfig(n_embd=1, block_size=1024, n_head=1, n_layer=4)

device = config.device

In [3]:
# Dataloading
import torch.utils


dir_path = os.path.dirname(os.path.abspath(os.getcwd()))
data_root = os.path.join(dir_path, "adl4cv", "datasets", "DWSNets", "mnist-inrs")

dataset = DWSNetsDataset(data_root)
path = "datasets/DWSNets/mnist-inrs/mnist_splits.json"
dataset_map = json.load(open(path, "r"))


In [4]:
flattened = dataset[0][0]

def get_batch(split: str):
    # let's get a batch with the single element
    # y should be the same shifted by 1
    ix = torch.zeros(config.batch_size, dtype=torch.int)
    #torch.randint(torch.numel(flattened) - model_config.block_size, (config.batch_size,))

    x = torch.stack(
        [flattened[i : i + model_config.block_size] for i in ix]
    )
    y = torch.stack(
        [flattened[i + 1 : i + 1 + model_config.block_size] for i in ix]
    )

    # x and y have to be (1, *, 1)
    x = x.unsqueeze(-1)
    y = y.unsqueeze(-1)
    return x, y

In [5]:
X, Y = get_batch("")
Y[0][-1]=12

model = RegressionTransformer(model_config)
logits, loss = model(X, Y)
print(Y[0][-1])
print(logits[0][-1])
print(loss)

tensor([12.])
tensor([-0.0001], grad_fn=<SelectBackward0>)
tensor(0.1411, grad_fn=<MseLossBackward0>)


In [6]:
print(logits[0][-1])

tensor([-0.0001], grad_fn=<SelectBackward0>)


In [7]:
# Prepeare model parameters and train
train(get_batch, config, model_config)

Initializing a new model from scratch
num decayed parameter tensors: 18, with 1,073 parameters
num non-decayed parameter tensors: 34, with 54 parameters


/Users/luis/uni/adl4cv/adl4cv/.venv/lib/python3.12/site-packages/torch/amp/grad_scaler.py:131: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/Users/luis/uni/adl4cv/adl4cv/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


using fused AdamW: False


wandb: Currently logged in as: luis-muschal (adl-for-cv). Use `wandb login --relogin` to force relogin


step 0: train loss 0.0004, val loss 0.0004
step 100: train loss 0.0004, val loss 0.0004
saving checkpoint to models
step 200: train loss 0.0004, val loss 0.0004
saving checkpoint to models
step 300: train loss 0.0004, val loss 0.0004
step 400: train loss 0.0004, val loss 0.0004
step 500: train loss 0.0004, val loss 0.0004
saving checkpoint to models
step 600: train loss 0.0004, val loss 0.0004
step 700: train loss 0.0004, val loss 0.0004
saving checkpoint to models
step 800: train loss 0.0004, val loss 0.0004
saving checkpoint to models
step 900: train loss 0.0004, val loss 0.0004
saving checkpoint to models
step 1000: train loss 0.0004, val loss 0.0004
saving checkpoint to models
step 1100: train loss 0.0004, val loss 0.0004
saving checkpoint to models
step 1200: train loss 0.0004, val loss 0.0004
saving checkpoint to models
step 1300: train loss 0.0004, val loss 0.0004
saving checkpoint to models
step 1400: train loss 0.0004, val loss 0.0004
saving checkpoint to models
step 1500: tra

wandb: Network error (ConnectionError), entering retry loop.


step 71400: train loss 0.0004, val loss 0.0004
saving checkpoint to models
step 71500: train loss 0.0004, val loss 0.0004
saving checkpoint to models
step 71600: train loss 0.0004, val loss 0.0004
saving checkpoint to models
step 71700: train loss 0.0004, val loss 0.0004
saving checkpoint to models
step 71800: train loss 0.0004, val loss 0.0004
saving checkpoint to models
step 71900: train loss 0.0004, val loss 0.0004
saving checkpoint to models
step 72000: train loss 0.0004, val loss 0.0004
saving checkpoint to models
step 72100: train loss 0.0004, val loss 0.0004
saving checkpoint to models
step 72200: train loss 0.0004, val loss 0.0004
saving checkpoint to models
step 72300: train loss 0.0004, val loss 0.0004
saving checkpoint to models
step 72400: train loss 0.0004, val loss 0.0004
saving checkpoint to models
step 72500: train loss 0.0004, val loss 0.0004
saving checkpoint to models
step 72600: train loss 0.0004, val loss 0.0004
saving checkpoint to models
step 72700: train loss 0.